## 1.1 Импорт библиотек и определение путей

Инициализация путей на основе настроей `config.py`. Если активирован `Сolab`

In [ ]:
import pandas as pd
import os
import sys

In [ ]:
# === Конфигурация ===
class Config:
  """
  Класс для хранения всех настроек проекта
  Если разработка происходит в Colab, изменить IS_COLAB = True
  """
  IS_COLAB = True
  DATA_DIR_NAME = "data"
  FILE_EXT = ".xlsx"
  WEEKS_RANGE = range(1, 21)

  # Путь к датасет файлам проекта
  DRIVE_PATH = ""

In [ ]:
if Config.IS_COLAB:
  try:
    from google.colab import drive
    drive.mount("/content/drive")

    BASE_DIR = os.path.join(Config.DRIVE_PATH, Config.DATA_DIR_NAME)
  except ImportError:
    print("Ошибка подключения к Google Drive")
else:
  BASE_DIR = Config.DATA_DIR_NAME

## 1.2 Загрузка данных

Объединяем еженедельные файлы в единый DataFrame. Имена файлов и количество недель генерируется на основе конфигурационного класса

In [ ]:
all_data = []
failed_files = []

for i in Config.WEEKS_RANGE:
  file_name = f'w{i}_25{Config.FILE_EXT}'
  file_path = os.path.join(BASE_DIR, file_name)

  try:
    df_chunk = pd.read_excel(file_path)
    all_data.append(df_chunk)
  except FileNotFoundError:
    failed_files.append(file_name)
  except Exception as e:
    print(f"Ошибка: {e}")

if len(failed_files) == 0:
  df = pd.concat(all_data, ignore_index=True)
else:
  print(f'Не удалось загрузить следующие файлы: {", ".join(failed_files)}')

## 1.3 Очистка и предобработка

1. Переименовываем столбцы в snake_case
2. Удаляем технические столбцы

In [ ]:
# Словарь маппинга для переименования столбцов
rename_map = {
    'Unnamed: 0': 'week',
    'Unnamed: 1': 'data',
    'Unnamed: 2': 'delete_1',
    'С': 'start_time',
    'По': 'end_time',
    'Часы': 'duration_hours',
    'Описание': 'description',
    'Финансовый Акт': 'delete_2',
    'Тип Активности':'type_of_activity',
    'Этап (опционально)': 'delete_3',
    'Начало оказания работ': 'delete_4',
    'Окончание работ': 'delete_5',
    'Сообщение об ошибке': 'delete_6',
    '#965':'delete_7',
    'Unnamed: 3':'delete_8',
    'Unnamed: 4':'delete_9',
    'Unnamed: 5':'delete_10',
    'Unnamed: 6':'delete_11',
    'Unnamed: 7':'delete_12',
    'Unnamed: 8':'delete_13',
    'Unnamed: 9':'delete_14',
    'Unnamed: 10':'delete_15',
    'Unnamed: 11':'delete_16',
    'Unnamed: 12':'delete_17',
}

# Переименование
df = df.rename(columns=rename_map)

In [ ]:
# Удаление
columns_to_drop = [col for col in df.columns if 'delete' in col]
df = df.drop(columns=columns_to_drop)

In [ ]:
df